## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
    - Done
* Apply a distortion correction to raw images.
    - Done
* Use color transforms, gradients, etc., to create a thresholded binary image.
    - Done
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

### FUNCTIONS

In [45]:
import cv2
import glob
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
%matplotlib inline

# need to find the transform from real world object coords to image coords
def findObjectPtsAndImgPts(pathToCalibImgs='./camera_cal/calibration*.jpg'
                              , shapeOfBoard=[9,6], verbose=False):
    """
    pathToCalibImgs: the regex to the calibration images that is passed to glob
    shapeOfBoard: the x and y dimensions of the chessboard used for calibration
    verbos: show the images and print data
    
    returns
    objPoints
    imgPoints
    """
    %matplotlib inline

    images = glob.glob(pathToCalibImgs)
    count = 0

    objPoints = [] # 3d points 
    imgPoints = [] # 2d points

    # we know the objects points because its a calibration image
    nx = shapeOfBoard[0] # interior chessboard corners in the x dim
    ny = shapeOfBoard[1] # interior chessboard corners in the y dim
    objPts = np.zeros((nx*ny, 3), np.float32)
    objPts[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

    for fileName in images:
        img = cv2.imread(fileName)
        # cv2 fn takes gray space img
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        # if the function finds chessboard corners, append known object points and found image points
        if ret:
            imgPoints.append(corners)
            objPoints.append(objPts)
            count = count + 1

            # visualize
            if verbose:
                cornersImg = cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
                plt.figure()
                plt.title(fileName)
                plt.imshow(cornersImg)
            
    if verbose:
        print('percentage found:', count/np.shape(images)[0] * 100)
        
    return objPoints, imgPoints
        
def calibrateCamera(objPoints, imgPoints, pathToImg='./camera_cal/calibration2.jpg'
                    , verbose=False):
    """
    wrapper for cv2.calibrateCamera, optional print images
    """
    
    %matplotlib inline

    img = cv2.imread(pathToImg)

    # cameraMtx, camera matrix that discribes the mapping of pinhole camera to convex camera pts
    # distortCoeffs, the tangential and radial distortion coefficients
    # rotVect, rotational vectors to get camera attitude
    # transVects, translational vecotrs to get camera position
    ret, cameraMtx, distortCoeffs, rotVects, transVects = \
        cv2.calibrateCamera(objPoints, imgPoints, img.shape[1::-1], None, None)

    if verbose:
        undistImg = cv2.undistort(img, cameraMtx, distortCoeffs, None, cameraMtx)

        # visualize
        plt.figure()
        plt.title('Undistorted image: \'%s\'' %pathToImg)
        plt.imshow(undistImg)
        
    return ret, cameraMtx, distortCoeffs, rotVects, transVects

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def roiMask(img, verbose=False):
    """
    inputs: 
      cannyImg, the result from performCannyEdgeDetection()
      show, plot the roi masked image
    outputs:
      roiImg, the resultant image
    """
#     img = cv2.imread(imagePath)
    xSize = img.shape[1]
    ySize = img.shape[0]
    
#     # mask inside of lane
#     roiInside = np.array([[(0,ySize),(xSize *(1/4), ySize),(xSize / 2, ySize * (2 / 3)),(xSize * (3/4),ySize),(xSize,ySize),(xSize,0),(0,0)]], dtype=np.int32)
#     roiImg = region_of_interest(img, roiInside)
    
    # mask outside of lane
    roi = np.array([[(0, ySize), (xSize, ySize), (xSize / 2 + (xSize / 20), ySize / 2 + (ySize / 10)), (xSize / 2 - (xSize / 20), ySize / 2 + (ySize / 10))]], dtype=np.int32)
    roiImg = region_of_interest(img, roi) # create empty mask, fill polygon in mask, bitwise and the mask and img
    
    if verbose:
        roiImage = plt.figure()
        plt.title('ROI Mask')
        plt.imshow(roiImg, cmap='gray')
        roiImage.savefig('./roiImg.jpg')
        
    return roiImg
    
def createBinary(imagePath, colorThresh=(170,255), xGradientThresh=(20,100)
                        , verbose=True):
    image = cv2.imread(imagePath)
    s_thresh = colorThresh # color channel threshhold
    sx_thresh = xGradientThresh # x gradient thresh hold

    img = np.copy(image)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # combine both thresholds into a binary
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    # Plot the result
    if verbose:
        f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
        f.tight_layout()

        ax1.imshow(image)
        ax1.set_title('Original Image', fontsize=40)

        ax2.imshow(color_binary)
        ax2.set_title('Sobel X, and Color channel ', fontsize=40)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

        ax3.imshow(combined_binary, cmap='gray')
        
    return combined_binary

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def findHoughLines(roiImg, show=False, rho=1, theta=(np.pi/360), threshold=15, min_line_len=12, max_line_gap=10):
    """
    inputs:
      roiImg, the result from performROIMasking()
      rho, the length resolution in a mesh unit [pixel]
      theta, the angular resolution of the mesh [radians]
      threshold, min number of votes for line detection
      min_line_len, [pixels]
      max_line_gap, [pixels]
      show, plot the hough line image
    outputs:
      houghImg, the resultant image
      lines, the resultant hough lines
    """
    lines = cv2.HoughLinesP(roiImg, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    houghImg = np.zeros((roiImg.shape[0], roiImg.shape[1], 3), dtype=np.uint8)
    draw_lines(houghImg, lines)
    if show:
        houghImage = plt.figure()
        plt.title('Hough Lines Img')
        plt.imshow(houghImg)
        houghImage.savefig('./HoughLines.jpg')
    return houghImg, lines

def findLeftAndRightLines(houghImg, lines, show=False):
    """
    inputs: 
      houghImg, the first resultant from findHoughLines()
      lines, the second resultant from findHoughLines()
      show, plot the left and right lane lines
    outputs:
      leftBin, the lines that constitute the left lane line
      rightBin, the lines that constitute the right lane line
    """
    xSize = houghImg.shape[1]
    ySize = houghImg.shape[0]
    leftBin = np.array([[0, ySize, 0, ySize]])
    rightBin = np.array([[xSize,ySize,xSize,ySize]])
    leftOrigins = []
    rightOrigins = []
        
    if show:
        plt.figure()
        plt.legend()
        
    for line in lines:
        for x1,y1,x2,y2 in line:
            lineInfo = linregress([x1, x2], [y1, y2])
            if lineInfo.slope == 0:
                continue
            elif abs(lineInfo.slope) >= np.pi/6 and (x1 <= xSize/2 and x2 <= xSize/2):
                if show:
                    plt.plot([x1, x2], [-y1, -y2], 'b')
                leftBin = np.append(leftBin, line, axis=0)
                leftOrigins = np.append(leftOrigins, ((ySize - lineInfo.intercept) / lineInfo.slope))
            elif abs(lineInfo.slope) >= np.pi/6 and (x1 > xSize/2 and x2 > xSize/2) :
                if show:
                    plt.plot([x1, x2], [-y1, -y2], 'r')
                rightBin = np.append(rightBin, line, axis=0)
                rightOrigins = np.append(rightOrigins, ((ySize - lineInfo.intercept) / lineInfo.slope))

    leftOrigin = np.median(leftOrigins)
    rightOrigin = np.median(rightOrigins)
    
    # sort the bins into tidy arrays
    leftBin[0,0] = leftBin[0,-2] = leftOrigin
    rightBin[0,0] = rightBin[0,-2] = rightOrigin
    
    leftVerts = np.array([leftBin[0, 0:2]])
    leftVerts = np.append(leftVerts, leftBin[1:, 0:2], axis=0)
    leftVerts = np.append(leftVerts, leftBin[:, 2:], axis=0)

    rightVerts = np.array([rightBin[0, 0:2]])
    rightVerts = np.append(rightVerts, rightBin[1:, 0:2], axis=0)
    rightVerts = np.append(rightVerts, rightBin[:, 2:], axis=0)
    
    if show:
        plt.title('Left and Right Lane Lines')
        plt.legend()
        plt.savefig('./LeftRightLanes.jpg')
        plt.show()
           
    return leftVerts, rightVerts 

### Testing

In [13]:
objPts, imgPts = findObjectPtsAndImgPts()
_ = calibrateCamera(objPts, imgPts, './camera_cal/calibration*.jpg', True)

AttributeError: 'NoneType' object has no attribute 'shape'

In [46]:
%matplotlib qt

images = glob.glob('./test_images/test[3-5].jpg')

for file in images:
    combined_binary = createBinary(file, (170,255), (20,100), False)
    roiImg = roiMask(combined_binary, False)
    
#     show=True
#     rho=1
#     theta=(np.pi/360)
#     threshold=15
#     min_line_len=50 
#     max_line_gap=10
#     houghImg, lines = findHoughLines(roiImg, show, rho, theta, threshold, min_line_len, max_line_gap)
#     leftVerts, rightVerts = findLeftAndRightLines(houghImg, lines, True)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
